In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import labolibrary as labo

from sklearn.preprocessing import MinMaxScaler



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "d:\Dropbox\Python\LaboIII\labo3-2024v\.venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\Dr

In [2]:

#DATOS_DIR = '~/buckets/b1/datasets/'
DATOS_DIR = '../data/'

# Definir la métrica personalizada
def multinacional_metric(y_true, y_pred):
    return abs(sum(y_true - y_pred)) / sum(y_true)
# Función para escalar y devolver una serie
def minmax_scale_group(group):
    scaler = MinMaxScaler()
    scaled_values = scaler.fit_transform(group.values.reshape(-1, 1)).flatten()
    scalers[group.name] = scaler  # Almacenar el escalador para este grupo
    return pd.Series(scaled_values, index=group.index)

# Función para desescalar y devolver una serie
def inverse_minmax_scale_group(group):
    scaler = scalers[group.name]
    inversed_values = scaler.inverse_transform(group.values.reshape(-1, 1)).flatten()
    return pd.Series(inversed_values, index=group.index)

# Leer datos
df_final = pd.read_parquet(DATOS_DIR+'FE_dataset-CARLA.parquet') 
df_final.columns = df_final.columns.str.replace(' ', '_').str.replace(r'[^A-Za-z0-9_]', '', regex=True)

### Filtrar datos
df_true = df_final.loc['2019-12-01':'2020-01-01']
df_final = df_final.loc['2018-01-01':'2019-10-01']


In [3]:

### escalar

scalers = {}

df_final['tn'] = df_final.groupby('product_id')['tn'].transform(minmax_scale_group) #escalado
df_final['tn_2'] = df_final.groupby('product_id')['tn_2'].transform(minmax_scale_group) #escalado


In [4]:
# Time Warping

dtw_features = labo.compute_dtw_features(df_final)

df_final = pd.concat([df_final, dtw_features], axis=1).fillna(0)


d:\Dropbox\Python\LaboIII\labo3-2024v\src\GradientBoosting2\NWO\labolibrary.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dtw_features[feature_name] = distance
d:\Dropbox\Python\LaboIII\labo3-2024v\src\GradientBoosting2\NWO\labolibrary.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dtw_features[feature_name] = distance
d:\Dropbox\Python\LaboIII\labo3-2024v\src\GradientBoosting2\NWO\labolibrary.py:90: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times,

In [ ]:

# Correr Modelo
params={
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
         'metric':'rmse',
        #'n_jobs': -1,
        #'seed': 113,
        #'learning_rate': 0.2,
        #'bagging_fraction': 0.85,
        #'bagging_freq': 1, 
        #'colsample_bytree': 0.85,
        #'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        #'lambda_l1': 0.5,
        #'lambda_l2': 0.5
}
model, average_metric = labo.train_lightgbm_model(df_final,params,metric='rmse')
print("Overall custom metric: ", average_metric)




In [ ]:
def predict_next_month(model, last_data_points):
    predictions = []
    last_month = last_data_points.index.max() + 1
    last_data_points.index = [last_month] * len(last_data_points)  # Set index to the next month
    
    predictions = model.predict(last_data_points, num_iteration=model.best_iteration)
    
    prediction_df = last_data_points[['product_id']].copy()
    prediction_df['tn_2'] = predictions
    prediction_df.index = [last_month] * len(last_data_points)

    return prediction_df

In [ ]:
# Predict values for the entire dataset using the trained models
# Prepare last data points for prediction
last_data_points = df_final[df_final.index == df_final.index.max()].copy()
last_data_points.drop(columns=['tn_2'], inplace=True)


In [ ]:
# Predict the next month's value using the trained model
predictions = predict_next_month(model, last_data_points)

preds = predictions.groupby('product_id')['tn_2'].transform(inverse_minmax_scale_group)
predictions['tn'] = preds
predictions.drop(columns=['tn_2'], inplace=True)
predictions = predictions.reset_index()
predictions =  predictions.groupby('product_id')['tn'].sum()
predictions.columns = ['product_id', 'tn']
predictions.to_csv(DATOS_DIR+'/pred/0005-prediccion-timewarping-rmse.csv', index=True,header=True)
print("Overall custom metric: ", average_metric)


In [ ]:

print("Error: " ,abs(sum(df_true.groupby('product_id')['tn'].sum().values-predictions.values))/sum(df_true.groupby('product_id')['tn'].sum().values))